In [85]:
import datetime
import hyperopt
import itertools
import numpy
import operator
import pandas
import requests
from IPython.display import Audio
from sklearn import (
    ensemble,
    linear_model,
    metrics,
    model_selection,
    naive_bayes,
    neighbors,
    neural_network,
    svm,
)
import time
import tqdm
import xgboost

### Read and Filter

In [3]:
dtype = {'smoke': float, 'alco': float, 'active': float}

def fix_ap(value):
    value = abs(value)
    while value > 500.0:
        value /= 10.0
    return value

def read_csv(filename):
    frame = pandas.read_csv(filename, sep=';', header=0, na_values='None', dtype=dtype).drop(['id'], axis=1)
    frame['ap_hi'] = frame['ap_hi'].apply(fix_ap)
    frame = pandas.get_dummies(frame, columns=[
        'gender',
        'cholesterol',
        'gluc',
        'smoke',
        'active',
        'alco',
    ])
    frame = frame.assign(
        # aged_smoke_0=(frame['smoke_0.0'] * frame['age']),
        # aged_smoke_1=(frame['smoke_1.0'] * frame['age']),
        # aged_active_0=(frame['active_0.0'] * frame['age']),
        # aged_active_1=(frame['active_1.0'] * frame['age']),
        # aged_alco_0=(frame['alco_0.0'] * frame['age']),
        # aged_alco_1=(frame['alco_1.0'] * frame['age']),
        # bmi=(frame['weight'] / frame['height'] ** 2),
        # smoke_1_active_0=(frame['smoke_1.0'] * frame['active_0.0']),
        # aged_cholesterol_1=(frame['cholesterol_1'] * frame['age']),
        # aged_cholesterol_2=(frame['cholesterol_2'] * frame['age']),
        # aged_cholesterol_3=(frame['cholesterol_3'] * frame['age']),
        # gender_1_smoke_1=(frame['gender_1'] * frame['smoke_1.0']),
        # gender_2_smoke_1=(frame['gender_2'] * frame['smoke_1.0']),
    )
    return frame

In [4]:
train = read_csv('train.csv')

X = train.drop('cardio', axis=1).values
y = train['cardio'].values
print(f'X: {X.shape}')
print(f'y: {y.shape}')

X: (70000, 19)
y: (70000,)


In [5]:
test = read_csv('test.csv')

X_test = test.values
print(f'X_test: {X_test.shape}')

X_test: (30000, 19)


In [6]:
train.head()

,age,height,weight,ap_hi,ap_lo,cardio,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,gluc_1,gluc_2,gluc_3,smoke_0.0,smoke_1.0,active_0.0,active_1.0,alco_0.0,alco_1.0
0,18393,168,62.0,110.0,80,0,0,1,1,0,0,1,0,0,1,0,0,1,1,0
1,20228,156,85.0,140.0,90,1,1,0,0,0,1,1,0,0,1,0,0,1,1,0
2,18857,165,64.0,130.0,70,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0
3,17623,169,82.0,150.0,100,1,0,1,1,0,0,1,0,0,1,0,0,1,1,0
4,17474,156,56.0,100.0,60,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0


In [7]:
test.head()

,age,height,weight,ap_hi,ap_lo,gender_1,gender_2,cholesterol_1,cholesterol_2,cholesterol_3,gluc_1,gluc_2,gluc_3,smoke_0.0,smoke_1.0,active_0.0,active_1.0,alco_0.0,alco_1.0
0,18888,154,85.0,130.0,80,1,0,1,0,0,1,0,0,1,0,0,1,1,0
1,19042,170,69.0,130.0,90,0,1,1,0,0,1,0,0,0,0,0,1,1,0
2,20432,160,70.0,120.0,75,1,0,1,0,0,1,0,0,1,0,1,0,1,0
3,18133,185,94.0,130.0,90,0,1,1,0,0,1,0,0,0,0,0,1,1,0
4,16093,170,76.0,120.0,80,0,1,1,0,0,1,0,0,1,0,0,1,1,0


### Predict and Save

In [65]:
def fit_predict_save(estimator, filename):
    estimator.fit(X, y)
    for name, importance in sorted(zip(test, estimator.feature_importances_), key=operator.itemgetter(1), reverse=True):
        print(f'{name}: {importance:.7f}')
    numpy.savetxt(filename, estimator.predict_proba(X_test)[:, 1], fmt='%f')

### XGBoost

In [104]:
def search_xgb(params, **param_grid):
    start_time = time.time()
    grid_search = model_selection.GridSearchCV(
        xgboost.XGBClassifier(nthread=5, seed=0, **params),
        param_grid,
        scoring='neg_log_loss',
        refit=False,
        cv=model_selection.StratifiedKFold(n_splits=3, shuffle=True, random_state=0),
    ).fit(X, y)
    print(f'[{(time.time() - start_time) / 60.0:.2f} min] {grid_search.best_score_:.7f} {grid_search.best_params_}')
    return grid_search

In [79]:
'''
xgb_params = {
    'n_estimators': 5, 
    'reg_lambda': 1.0,
    'scale_pos_weight': 1.0,
    'base_score': 0.5,
}
'''

In [105]:
def make_int_range(params, param_name, delta):
    n = params.get(param_name, 1)
    return {param_name: range(max(1, n - delta), n + delta + 1)}

def make_float_range(params, param_name, delta, min_, max_):
    x = params.get(param_name, 0.0)
    return {param_name: [max(min_, x - delta), x, min(max_, x + delta)]}

for _ in itertools.count(start=1):
    last_params = dict(xgb_params)
    xgb_params.update(search_xgb(xgb_params, **make_float_range(xgb_params, 'base_score', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_range(xgb_params, 'scale_pos_weight', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_range(xgb_params, 'reg_lambda', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_range(xgb_params, 'reg_alpha', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_float_range(xgb_params, 'gamma', 0.01, 0.0, 1.0)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_int_range(xgb_params, 'max_depth', 1)).best_params_)
    xgb_params.update(search_xgb(xgb_params, **make_int_range(xgb_params, 'n_estimators', 3)).best_params_)
    if xgb_params == last_params:
        break

[0.12 min] -0.5390210 {'base_score': 0.51}
[0.12 min] -0.5390210 {'scale_pos_weight': 0.99}
[0.13 min] -0.5390210 {'reg_lambda': 0.99}
[0.13 min] -0.5390210 {'reg_alpha': 0.02}
[0.13 min] -0.5390210 {'gamma': 0.19}
[0.13 min] -0.5390210 {'max_depth': 4}
[0.30 min] -0.5389971 {'n_estimators': 86}
[0.13 min] -0.5389971 {'base_score': 0.51}
[0.13 min] -0.5389971 {'scale_pos_weight': 0.99}
[0.13 min] -0.5389971 {'reg_lambda': 0.99}
[0.12 min] -0.5389971 {'reg_alpha': 0.02}
[0.13 min] -0.5389971 {'gamma': 0.19}
[0.12 min] -0.5389971 {'max_depth': 4}
[0.29 min] -0.5389719 {'n_estimators': 89}
[0.13 min] -0.5389719 {'base_score': 0.51}
[0.13 min] -0.5389719 {'scale_pos_weight': 0.99}
[0.13 min] -0.5389719 {'reg_lambda': 0.99}
[0.13 min] -0.5389719 {'reg_alpha': 0.02}
[0.13 min] -0.5389719 {'gamma': 0.19}
[0.13 min] -0.5389719 {'max_depth': 4}
[0.30 min] -0.5389624 {'n_estimators': 90}
[0.13 min] -0.5389624 {'base_score': 0.51}
[0.13 min] -0.5389624 {'scale_pos_weight': 0.99}
[0.13 min] -0.538

In [106]:
fit_predict_save(xgboost.XGBClassifier(nthread=5, **xgb_params), f'xgboost.txt')

age: 0.2540416
ap_hi: 0.1739800
weight: 0.1339492
height: 0.0993072
ap_lo: 0.0931486
cholesterol_1: 0.0446497
cholesterol_3: 0.0369515
gluc_3: 0.0331024
active_0.0: 0.0331024
smoke_0.0: 0.0192456
alco_0.0: 0.0192456
gender_1: 0.0184758
gluc_1: 0.0169361
cholesterol_2: 0.0138568
gluc_2: 0.0100077
gender_2: 0.0000000
smoke_1.0: 0.0000000
active_1.0: 0.0000000
alco_1.0: 0.0000000
